# EXAMPLE GET INLIERS
In this notebook, we ...
As ouput, the method generates a selection of nodes 

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [23]:
#IMPORT PACKAGES
import rdflib
import os, time
import importlib
import numpy as np
import open3d as o3d
import ifcopenshell
import unittest

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
from geomapi.nodes import ImageNode, PointCloudNode, MeshNode, BIMNode, Node
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
import geomapi.tools.progresstools as pgt
import unittest


from rdflib import RDF, Graph, Namespace, Literal, URIRef

In [24]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
%autoreload 2

## 1. INPUT

In [27]:

pcd=o3d.io.read_point_cloud('../tests/testfiles/pcd/parking.pcd')
pcdNode = PointCloudNode(resource=pcd) # built from resource or data
print(pcdNode.resource)
meshNode= MeshNode (path='../tests/testfiles/mesh/parking.obj') # .stl and .obj are supported
print(meshNode.resource)
imgNode=ImageNode(xmpPath='../tests/testfiles/img/DJI_0085.xmp') # .xmp contains pose information from CapturingReality software. MetaShape .xml is also supported.
print(imgNode.cartesianTransform)

# tl.select_nodes_k_nearest_neighbors(pcdNode,[meshNode,imgNode],k=1) #selects the k nearest neighbors of a point cloud node from a list of nodes
# imgNode.create_rays(imagePoints=[[0,0],[0,1],[1,0],[1,1]],depths=25) #creates rays from image points

import ifcopenshell.util.selector
ifc = ifcopenshell.open('../tests/testfiles/ifc/road.ifc')  
# element=ifc.by_guid('2PpmXyFnz1_QUVtw0_xe4L') 

# node= BIMNode(ifcPath='../tests/testfiles/ifc/parking.ifc',getResource=True)  
print(ifcopenshell.util.selector.filter_elements(ifc,"IfcSite"))    
ifcElement=next((ifcElement for ifcElement in ifcopenshell.util.selector.filter_elements(ifc,"IfcElement")),None)
  
# classes= 'IfcBeam,IfcColumn,IfcWall,IfcSlab'
# elements=ifcopenshell.util.selector.filter_elements(ifc,classes)
print(node.resource)
# elements=ifcopenshell.util.selector.filter_elements(self.dataLoaderParking.ifc, 'ifcObject')

PointCloud with 556485 points.
TriangleMesh with 62135 points and 49999 triangles.
[[-8.13902571e-02  6.83059476e-01 -7.25813597e-01  5.18276221e+01]
 [ 9.96648497e-01  4.97790854e-02 -6.49139139e-02  6.10007435e+01]
 [-8.20972697e-03 -7.28664391e-01 -6.84821733e-01  1.50408221e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
{#7956=IfcSite('3H7DyiHwT7hwR69f2l72fp',#2,'BT9_Deksel_Prive_Huisaansluiting_WPI22',$,$,#7957,#7958,$,.PARTIAL.,$,$,$,$,$), #14257=IfcSite('3dzlFaOIb0bx0z6uxD96Sz',#2,'BT1_Bodembedekking_WSV10',$,$,#14258,#14259,$,.PARTIAL.,$,$,$,$,$), #7850=IfcSite('0LQpWG_GX4rA6NvvP3ddlP',#2,'BT9_Deksel_Waterafvoer_Rond_WRI1',$,$,#7851,#7852,$,.PARTIAL.,$,$,$,$,$), #7940=IfcSite('0iHQz$0bP5KgZgVt8tDVtX',#2,'BT9_Deksel_Prive_Huisaansluiting_WPI22',$,$,#7941,#7942,$,.PARTIAL.,$,$,$,$,$), #7916=IfcSite('0TIlRB$hX9cOF3knr1ty7Y',#2,'BT9_Deksel_Prive_Huisaansluiting_WPI22',$,$,#7917,#7918,$,.PARTIAL.,$,$,$,$,$), #6776=IfcSite('2NYKFQWFP12wHMSLLUof9b',#2,'BT1_On

In [7]:
path= os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),"test","testfiles" )#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 

#GRAPH testNodeGraph.ttl
testNodeGraphPath=os.path.join(path,'testNodeGraph.ttl')
testNodeGraph=Graph().parse(testNodeGraphPath)

#GRAPH bimGraph.ttl
bimGraphPath=os.path.join(path,'bimGraph.ttl')
bimGraph=Graph().parse(bimGraphPath)

#GRAPH testGraph.ttl
testGraphPath=os.path.join(path,'testGraph.ttl')
testGraph=Graph().parse(testGraphPath)
print(len(testGraph))

#GRAPH imgGraph.ttl
imgGraphPath=os.path.join(path,'imgGraph.ttl')
imgGraph=Graph().parse(imgGraphPath)
    

resourcePath=os.path.join(path,'resources')
tempGraphPath=os.path.join(resourcePath,'myNode.ttl')
if not os.path.exists(resourcePath):
    os.mkdir(resourcePath)

c:\Users\u0094523\code\geomapi\test\testfiles\testNodeGraph.ttl does not look like a valid URI, trying to serialize this will break.


URLError: <urlopen error unknown url type: c>

In [117]:
exif = rdflib.Namespace('http://www.w3.org/2003/12/exif/ns#')
geo=rdflib.Namespace('http://www.opengis.net/ont/geosparql#') #coordinate system information
gom=rdflib.Namespace('https://w3id.org/gom#') # geometry representations => this is from mathias
omg=rdflib.Namespace('https://w3id.org/omg#') # geometry relations
fog=rdflib.Namespace('https://w3id.org/fog#')
v4d=rdflib.Namespace('https://w3id.org/v4d/core#')
openlabel=rdflib.Namespace('https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f#')
e57=rdflib.Namespace('http://libe57.org#')
xcr=rdflib.Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#')
ifc=rdflib.Namespace('http://ifcowl.openbimstandards.org/IFC2X3_Final#')


In [130]:
  #graphPath and subject
node=Node('myNode',graphPath=bimGraphPath)
print(node.path)
print(node.get_path())
# self.assertEqual(node.path,os.path.join(ut.get_folder(self.bimGraphPath),node.name + '.txt'))
with open(bimGraphPath, 'w') as f:
            # base=ut.get_folder(self.graphPath)
    node.graph.serialize(f)#,base=base
                # if os.path.exists(f):
                #     return True
                # return False


None
d:\Scan-to-BIM repository\geomapi\test\testfiles\myNode.txt


TypeError: write() argument must be str, not bytes

In [91]:
testnode=Node(graphPath=tempGraphPath)
print(testnode.subject)
print(testnode.className)
print(testnode.subject)
print(testnode.subject)
print(testnode.subject)


AttributeError: 'Node' object has no attribute '_graph'

In [25]:
string='d:\\Scan-to-BIM repository\\geomapi\\test\\testfiles\\resources'
test=ut.get_folder(string)
print(test)
path='..\\ifcfile.ifc'
qsdf=path.strip('..\\')
qsdf=os.path.abspath(path)
print(qsdf)
print(os.path.join(test,qsdf))

D:\Scan-to-BIM repository\geomapi\test\testfiles
d:\Scan-to-BIM repository\geomapi\ifcfile.ifc
d:\Scan-to-BIM repository\geomapi\ifcfile.ifc


In [6]:
#node with only subject
subject='myNode'
testpath=os.path.join(resourcePath,'myNode.ttl')
myNode= Node(subject=subject,myFloat=0.1)
myNode.to_graph(testpath,save=True)
testnode=Node(graphPath=testpath)#graphPath=testpath
print(testnode.subject)
print(myNode.subject)


: 

: 

In [9]:
testpath=os.path.join(resourcePath,'myNode.ttl')

#node with a graph and some kwargs
test = Graph().parse(testpath)
myNode= Node(graph=test,myNewRemark='myNewRemark',myFloat=0.1)        
myNode.to_graph(testpath,save=True)
testnode=Node(graphPath=testpath)

print(testnode.subject)
print(myNode.subject)
print(testnode.graph.serialize())

file:///myNode
file:///myNode
@prefix v4d: <https://w3id.org/v4d/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<file:///myNode> a v4d:Node ;
    v4d:myFloat "0.1"^^xsd:float ;
    v4d:myNewRemark "myNewRemark" ;
    v4d:name "myNode" .




In [12]:
graph=imgGraph
graphPath=imgGraphPath
subject=[s for s in graph.subjects() if '101_0366_0036' in s]
node= Node(subject=subject[0],graphPath=graphPath)
node.to_graph()
print(node.graph.serialize())
# assertEqual(node.subject.toPython(),subject.toPython())
object=node.graph.value(subject,node.graph.exif['imageHeight'])
print(object)
# assertEqual(node.imageHeight,object.toPython())

@prefix e57: <http://libe57.org#> .
@prefix exif: <http://www.w3.org/2003/12/exif/ns#> .
@prefix v4d: <https://w3id.org/v4d/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<file:///d:/Scan-to-BIM%20repository/geomapi/test/testfiles/101_0366_0036> a "<class 'geomapi.nodes.imagenode.ImageNode'>" ;
    e57:cartesianTransform """[[-9.01077021e-01 -4.33659085e-01  0.00000000e+00  6.00575993e+05]
 [ 4.33659085e-01 -9.01077021e-01  0.00000000e+00  6.96281654e+05]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  3.13845765e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]""" ;
    exif:imageHeight "3648.0"^^xsd:float ;
    exif:imageWidth "5472.0"^^xsd:float ;
    v4d:name "101_0366_0036" ;
    v4d:path "..\\..\\tests\\Samples10\\101_0366_0036.jpg" ;
    v4d:sxy "0.02" ;
    v4d:sz "0.05" ;
    v4d:xmlIndex "0" ;
    v4d:xmlPath "..\\..\\tests\\Samples10\\ReferenceLAMBERT08_TAW.xml" .




AttributeError: 'Graph' object has no attribute 'exif'

In [11]:
graph=testNodeGraph
graphPath=testNodeGraphPath
subject=next(graph.subjects())
node= Node(graphPath=graphPath)
node.to_graph(testpath,save=True)
print(subject)
print(node.subject)

print(node.name)
print(graph.value(subject,RDFS.label))
# assertEqual(node.subject.toPython(),subject.toPython())
# object=graph.value(subject,RDFS.label)
# assertEqual(node.name,object.toPython())


http://IMG_2173
http://IMG_2173
IMG_2173
{b9c3f2e1-aea0-11ec-b2d9-a86daaa4a624}


In [13]:
#test save
newGraphPath=os.path.join(path,'resources','testGraph.ttl')
node=Node(graph=testGraph)
node.to_graph(graphPath=newGraphPath,save=True)     

# assertTrue(os.path.exists(newGraphPath))
# newGraph=Graph().parse(newGraphPath)
# assertEqual(len(newGraph),len(node.graph))

<Graph identifier=N3d8027e746604e07825fa7c3412b9785 (<class 'rdflib.graph.Graph'>)>

In [111]:
value=np.array([1,2,3])
dataType=ut.get_data_type(value)
test=Literal(value)#,datatype=dataType
print(test.toPython())
print(type(test.toPython()))

[1 2 3]
<class 'numpy.ndarray'>


In [22]:
newGraph=Graph()
value=0.1
dataType=ut.get_data_type(value)
v4d=rdflib.Namespace('https://w3id.org/v4d/core#')
sub=URIRef('http://myNode')
newGraph.add((sub, v4d['type'], Literal('qsdfsdqf') ))
newGraph.add((sub, v4d['test'], Literal(1,datatype=dataType)))
newGraph.serialize(testpath)



<Graph identifier=Nce348909082040f2b264ba63d2442c36 (<class 'rdflib.graph.Graph'>)>

In [37]:
test=Graph().parse(testpath)
v4d=rdflib.Namespace('https://w3id.org/v4d/core#')

qsdf=test.value(next(test.subjects()),v4d['myFloat'])
uytr=qsdf.toPython()
print(type(uytr))

<class 'float'>
